In [15]:
from pathlib import Path

import s3fs

from expression_atlas_db import base, load_db, settings, queries, utils

### Generate test datasets/gtf

In [48]:
!grep chr22 ../../veliadb_v0c.gtf > ../expression_atlas_db/test/test_data/veliadb_test.gtf

In [ ]:
!grep chr22 ../../veliadb_v2.gtf > ../expression_atlas_db/test/test_data/veliadb_test_v2.gtf

In [4]:
gtf = utils.GTFParser('../expression_atlas_db/test/test_data/veliadb_test.gtf')

In [5]:
gtf.transcript_df

,transcript_id,gene_id,veliadb_id,gene_biotype,assembly_id
0,ENST00000624155.2,ENSG00000279973.2,198908,lncRNA,test
1,ENST00000629028.1,ENSG00000281548.1,202998,lncRNA,test
2,ENST00000455843.5,ENSG00000284874.1,69813,protein_coding,test
3,ENST00000431044.5,ENSG00000284874.1,52474,protein_coding,test
4,ENST00000424559.2,ENSG00000235776.2,47915,processed_pseudogene,test
...,...,...,...,...,...
9106,CHS.35618.27,3985,465252,protein_coding,test
9107,rna-NM_001135911.1,113791,276606,protein_coding,test
9108,rna-NR_120386.1,54944,328109,lncRNA,test
9109,CHS.35626.11,4733,465253,protein_coding,test


In [6]:
gtf.gene_df

,gene_id,veliadb_id,gene_biotype,assembly_id
0,ENSG00000279973.2,12509,lncRNA,test
1,ENSG00000281548.1,16346,lncRNA,test
2,ENSG00000284874.1,16424,protein_coding,test
3,ENSG00000235776.2,16463,processed_pseudogene,test
4,ENSG00000289960.1,16484,lncRNA,test
...,...,...,...,...
2114,55000,1399891,protein_coding,test
2115,3985,1399962,protein_coding,test
2116,113791,1399978,protein_coding,test
2117,54944,1399992,lncRNA,test


In [16]:
exp_srp129004 = utils.ExperimentParser('SRP129004', Path(settings.s3_experiment_loc))

In [45]:
exp_erp106487 = utils.ExperimentParser('ERP106487', Path(settings.s3_experiment_loc))

In [17]:
s3 = s3fs.S3FileSystem()

In [18]:
exp_srp129004.enable_s3(s3)

In [46]:
exp_erp106487.enable_s3(s3)

In [20]:
exp_srp129004.load_adatas()

In [47]:
exp_erp106487.load_adatas()

In [33]:
exp_srp129004._adata_transcript = exp_srp129004._adata_transcript[:,exp_srp129004._adata_transcript.var.index.isin(gtf.transcript_df['transcript_id'])]
display(exp_srp129004._adata_transcript)
exp_srp129004._adata_gene = exp_srp129004._adata_gene[:,exp_srp129004._adata_gene.var.index.isin(gtf.gene_df['gene_id'])]
display(exp_srp129004._adata_gene)

View of AnnData object with n_obs × n_vars = 222 × 3955
    obs: 'condition-2', 'condition-1', 'sample_type_2', 'sample_type_1', 'sample_condition_2', 'sample_condition_1', 'disease_1', 'single_end', 'strandedness', 'experiment_accession', 'submission_accession', 'library_layout', 'library_selection', 'library_source', 'library_strategy', 'library_name', 'instrument_model', 'instrument_platform', 'read_count', 'tax_id', 'sample_title', 'experiment_title', 'sample_description', 'size_factors', 'lib_sizes', 'sample_sums'
    var: 'length', 'mean', 'std'
    uns: '_squared_logres', 'condition-1_colors', 'condition-2_colors', 'contrasts', 'gene_transcript_mapping', 'instrument_model_colors', 'library_layout_colors', 'normed_counts_transform_scalefactor', 'pca', 'prior_disp_var', 'stat_results', 'trend_coeffs'
    obsm: 'X_pca', 'design_matrix', 'replaceable', 'size_factors'
    varm: 'LFC', 'LFC_reflevel_CONTROL', 'LFC_reflevel_CONTROL_raw', 'MAP_dispersions', 'PCs', '_LFC_converged', '_MA

View of AnnData object with n_obs × n_vars = 222 × 1022
    obs: 'condition-2', 'condition-1', 'sample_type_2', 'sample_type_1', 'sample_condition_2', 'sample_condition_1', 'disease_1', 'single_end', 'strandedness', 'experiment_accession', 'submission_accession', 'library_layout', 'library_selection', 'library_source', 'library_strategy', 'library_name', 'instrument_model', 'instrument_platform', 'read_count', 'tax_id', 'sample_title', 'experiment_title', 'sample_description', 'size_factors', 'lib_sizes', 'sample_sums'
    var: 'length', 'mean', 'std'
    uns: '_squared_logres', 'condition-1_colors', 'condition-2_colors', 'contrasts', 'gene_transcript_mapping', 'gsea_gene_df', 'gsea_results_prerank', 'gsea_results_standard', 'instrument_model_colors', 'library_layout_colors', 'normed_counts_transform_scalefactor', 'pca', 'prior_disp_var', 'ssgsea_colnames', 'stat_results', 'stat_results_gsea', 'trend_coeffs'
    obsm: 'KEGG_2016_ssgsea_es', 'KEGG_2016_ssgsea_nes', 'Reactome_2016_ssgsea

In [49]:
exp_erp106487._adata_transcript = exp_erp106487._adata_transcript[:,exp_erp106487._adata_transcript.var.index.isin(gtf.transcript_df['transcript_id'])]
display(exp_srp129004._adata_transcript)
exp_erp106487._adata_gene = exp_erp106487._adata_gene[:,exp_erp106487._adata_gene.var.index.isin(gtf.gene_df['gene_id'])]
display(exp_erp106487._adata_gene)

View of AnnData object with n_obs × n_vars = 222 × 3955
    obs: 'condition-2', 'condition-1', 'sample_type_2', 'sample_type_1', 'sample_condition_2', 'sample_condition_1', 'disease_1', 'single_end', 'strandedness', 'experiment_accession', 'submission_accession', 'library_layout', 'library_selection', 'library_source', 'library_strategy', 'library_name', 'instrument_model', 'instrument_platform', 'read_count', 'tax_id', 'sample_title', 'experiment_title', 'sample_description', 'size_factors', 'lib_sizes', 'sample_sums'
    var: 'length', 'mean', 'std'
    uns: '_squared_logres', 'condition-1_colors', 'condition-2_colors', 'contrasts', 'gene_transcript_mapping', 'instrument_model_colors', 'library_layout_colors', 'normed_counts_transform_scalefactor', 'pca', 'prior_disp_var', 'stat_results', 'trend_coeffs'
    obsm: 'X_pca', 'design_matrix', 'replaceable', 'size_factors'
    varm: 'LFC', 'LFC_reflevel_CONTROL', 'LFC_reflevel_CONTROL_raw', 'MAP_dispersions', 'PCs', '_LFC_converged', '_MA

View of AnnData object with n_obs × n_vars = 79 × 859
    obs: 'condition-4', 'condition-3', 'condition-2', 'condition-1', 'sample_type_2', 'sample_type_1', 'sample_condition_5', 'sample_condition_4', 'sample_condition_3', 'sample_condition_2', 'sample_condition_1', 'disease_2', 'disease_1', 'single_end', 'strandedness', 'submission_accession', 'library_layout', 'library_selection', 'library_source', 'library_strategy', 'library_name', 'instrument_model', 'instrument_platform', 'read_count', 'tax_id', 'sample_title', 'experiment_title', 'size_factors', 'lib_sizes', '-condition-1', 'sample_sums'
    var: 'length', 'mean', 'std'
    uns: '_squared_logres', 'condition-1_colors', 'condition-2_colors', 'condition-3_colors', 'condition-4_colors', 'contrasts', 'gene_transcript_mapping', 'gsea_gene_df', 'gsea_results_prerank', 'gsea_results_standard', 'instrument_model_colors', 'library_layout_colors', 'normed_counts_transform_scalefactor', 'pca', 'prior_disp_var', 'sample_type_1_colors', 'sam

In [51]:
for k,v in exp_srp129004._adata_transcript.uns['stat_results'].items():
    exp_srp129004._adata_transcript.uns['stat_results'][k] = v[v.index.isin(gtf.transcript_df['transcript_id'])]
    print(exp_srp129004._adata_transcript.uns['stat_results'][k].shape)
for k,v in exp_srp129004._adata_gene.uns['stat_results'].items():
    exp_srp129004._adata_gene.uns['stat_results'][k] = v[v.index.isin(gtf.gene_df['gene_id'])]
    print(exp_srp129004._adata_gene.uns['stat_results'][k].shape)


(3955, 232)
(1022, 231)


In [50]:
for k,v in exp_erp106487._adata_transcript.uns['stat_results'].items():
    exp_erp106487._adata_transcript.uns['stat_results'][k] = v[v.index.isin(gtf.transcript_df['transcript_id'])]
    print(exp_erp106487._adata_transcript.uns['stat_results'][k].shape)
for k,v in exp_erp106487._adata_gene.uns['stat_results'].items():
    exp_erp106487._adata_gene.uns['stat_results'][k] = v[v.index.isin(gtf.gene_df['gene_id'])]
    print(exp_erp106487._adata_gene.uns['stat_results'][k].shape)


(3580, 20)
(3580, 20)
(3580, 20)
(3580, 62)
(3580, 31)
(3580, 32)
(3580, 31)
(3580, 31)
(3580, 32)
(3580, 31)
(3580, 64)
(3580, 62)
(859, 19)
(859, 19)
(859, 19)
(859, 61)
(859, 30)
(859, 31)
(859, 30)
(859, 30)
(859, 31)
(859, 30)
(859, 63)
(859, 61)


In [42]:
exp_srp129004._adata_gene.write_h5ad('../expression_atlas_db/test/test_data/test/SRP129004_test/de_results/SRP129004_test_dds_gene.h5ad')
exp_srp129004._adata_transcript.write_h5ad('../expression_atlas_db/test/test_data/test/SRP129004_test/de_results/SRP129004_test_dds_transcript.h5ad')

In [52]:
exp_erp106487._adata_gene.write_h5ad('../expression_atlas_db/test/test_data/test/ERP106487_test/de_results/ERP106487_test_dds_gene.h5ad')
exp_erp106487._adata_transcript.write_h5ad('../expression_atlas_db/test/test_data/test/ERP106487_test/de_results/ERP106487_test_dds_transcript.h5ad')

In [67]:
exp_srp129004_test = utils.ExperimentParser('SRP129004_test', Path('../expression_atlas_db/test/test_data/test'))
exp_srp129004_test.load_adatas()
display(exp_srp129004_test._adata_gene)
display(exp_srp129004_test._adata_transcript)

AnnData object with n_obs × n_vars = 222 × 1022 backed at '../expression_atlas_db/test/test_data/SRP129004_test/de_results/SRP129004_test_dds_gene.h5ad'
    obs: 'condition-2', 'condition-1', 'sample_type_2', 'sample_type_1', 'sample_condition_2', 'sample_condition_1', 'disease_1', 'single_end', 'strandedness', 'experiment_accession', 'submission_accession', 'library_layout', 'library_selection', 'library_source', 'library_strategy', 'library_name', 'instrument_model', 'instrument_platform', 'read_count', 'tax_id', 'sample_title', 'experiment_title', 'sample_description', 'size_factors', 'lib_sizes', 'sample_sums'
    var: 'length', 'mean', 'std'
    uns: '_squared_logres', 'condition-1_colors', 'condition-2_colors', 'contrasts', 'gene_transcript_mapping', 'gsea_gene_df', 'gsea_results_prerank', 'gsea_results_standard', 'instrument_model_colors', 'library_layout_colors', 'normed_counts_transform_scalefactor', 'pca', 'prior_disp_var', 'ssgsea_colnames', 'stat_results', 'stat_results_gse

AnnData object with n_obs × n_vars = 222 × 3955 backed at '../expression_atlas_db/test/test_data/SRP129004_test/de_results/SRP129004_test_dds_transcript.h5ad'
    obs: 'condition-2', 'condition-1', 'sample_type_2', 'sample_type_1', 'sample_condition_2', 'sample_condition_1', 'disease_1', 'single_end', 'strandedness', 'experiment_accession', 'submission_accession', 'library_layout', 'library_selection', 'library_source', 'library_strategy', 'library_name', 'instrument_model', 'instrument_platform', 'read_count', 'tax_id', 'sample_title', 'experiment_title', 'sample_description', 'size_factors', 'lib_sizes', 'sample_sums'
    var: 'length', 'mean', 'std'
    uns: '_squared_logres', 'condition-1_colors', 'condition-2_colors', 'contrasts', 'gene_transcript_mapping', 'instrument_model_colors', 'library_layout_colors', 'normed_counts_transform_scalefactor', 'pca', 'prior_disp_var', 'stat_results', 'trend_coeffs'
    obsm: 'X_pca', 'design_matrix', 'replaceable', 'size_factors'
    varm: 'LFC

In [68]:
exp_erp106487_test = utils.ExperimentParser('ERP106487_test', Path('../expression_atlas_db/test/test_data/test'))
exp_erp106487_test.load_adatas()
display(exp_erp106487_test._adata_gene)
display(exp_erp106487_test._adata_transcript)

AnnData object with n_obs × n_vars = 79 × 859 backed at '../expression_atlas_db/test/test_data/ERP106487_test/de_results/ERP106487_test_dds_gene.h5ad'
    obs: 'condition-4', 'condition-3', 'condition-2', 'condition-1', 'sample_type_2', 'sample_type_1', 'sample_condition_5', 'sample_condition_4', 'sample_condition_3', 'sample_condition_2', 'sample_condition_1', 'disease_2', 'disease_1', 'single_end', 'strandedness', 'submission_accession', 'library_layout', 'library_selection', 'library_source', 'library_strategy', 'library_name', 'instrument_model', 'instrument_platform', 'read_count', 'tax_id', 'sample_title', 'experiment_title', 'size_factors', 'lib_sizes', '-condition-1', 'sample_sums'
    var: 'length', 'mean', 'std'
    uns: '_squared_logres', 'condition-1_colors', 'condition-2_colors', 'condition-3_colors', 'condition-4_colors', 'contrasts', 'gene_transcript_mapping', 'gsea_gene_df', 'gsea_results_prerank', 'gsea_results_standard', 'instrument_model_colors', 'library_layout_colo

AnnData object with n_obs × n_vars = 79 × 3580 backed at '../expression_atlas_db/test/test_data/ERP106487_test/de_results/ERP106487_test_dds_transcript.h5ad'
    obs: 'condition-4', 'condition-3', 'condition-2', 'condition-1', 'sample_type_2', 'sample_type_1', 'sample_condition_5', 'sample_condition_4', 'sample_condition_3', 'sample_condition_2', 'sample_condition_1', 'disease_2', 'disease_1', 'single_end', 'strandedness', 'submission_accession', 'library_layout', 'library_selection', 'library_source', 'library_strategy', 'library_name', 'instrument_model', 'instrument_platform', 'read_count', 'tax_id', 'sample_title', 'experiment_title', 'size_factors', 'lib_sizes', '-condition-1', 'sample_sums'
    var: 'length', 'mean', 'std'
    uns: '_squared_logres', 'condition-1_colors', 'condition-2_colors', 'condition-3_colors', 'condition-4_colors', 'contrasts', 'gene_transcript_mapping', 'instrument_model_colors', 'library_layout_colors', 'normed_counts_transform_scalefactor', 'pca', 'prior_

In [1]:
!tar -C ../expression_atlas_db/test/ -czvf ../expression_atlas_db/test/test_data.tar.gz ./test_data/ 

./test_data/
./test_data/test.gtf
./test_data/ERP106487/
./test_data/ERP106487/de_results/
./test_data/ERP106487/de_results/ERP106487_test_dds_gene.h5ad
./test_data/ERP106487/de_results/ERP106487_test_dds_transcript.h5ad
./test_data/SRP129004/
./test_data/SRP129004/de_results/
./test_data/SRP129004/de_results/SRP129004_test_dds_gene.h5ad
./test_data/SRP129004/de_results/SRP129004_test_dds_transcript.h5ad
./test_data/test_v2.gtf
./test_data/test/
./test_data/test/SRP129004/
./test_data/test/SRP129004/de_results/
./test_data/test/SRP129004/de_results/SRP129004_test_dds_gene.h5ad
./test_data/test/SRP129004/de_results/SRP129004_test_dds_transcript.h5ad
./test_data/test/ERP106487/
./test_data/test/ERP106487/de_results/
./test_data/test/ERP106487/de_results/ERP106487_test_dds_gene.h5ad
./test_data/test/ERP106487/de_results/ERP106487_test_dds_transcript.h5ad
./test_data/veliadb_test.gtf
./test_data/veliadb_test_v2.gtf
